# TP3:
### Autores: Keimel, Cristian; Merlo, Pablo; Gleser Matias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [32]:
pip install lightfm

### importamos librerias

In [33]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank
import random
import pickle

# Load data
anime_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TP3/anime.csv')
ratings_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TP3/animelist.csv')

ratings_data.user_id.nunique()

CPU times: user 26.4 s, sys: 11.2 s, total: 37.6 s
Wall time: 56.7 s


325770

In [34]:
anime_data.head(3)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0


In [35]:
#Cuantos usuarios tenemos?
ratings_data.user_id.nunique()

325770

In [36]:
#Cuantos animes tenemos?
ratings_data.anime_id.nunique()

17562

In [37]:
#en el Df 'anime_data' coincide la cant. de animes?
anime_data.MAL_ID.nunique()

17562

### Preprocess data

In [38]:

random.seed(42)
selected_users = random.sample(list(ratings_data['user_id'].unique()), k= 5000)

ratings_data = ratings_data[ratings_data['user_id'].isin(selected_users)]
ratings_data = ratings_data.dropna()

In [24]:
ratings_data.user_id.nunique()

5000

In [39]:
#top30 animes mejor calificados en promedio
top30= ratings_data.groupby('anime_id').rating.mean().sort_values(ascending=False)[0:30]
top30

anime_id
40062    7.714286
42076    7.166667
39164    7.000000
2904     6.996165
38524    6.992958
199      6.984811
32281    6.931304
1535     6.930155
30276    6.830803
32935    6.816162
16498    6.785628
39163    6.750000
431      6.733154
28851    6.721075
42984    6.692308
41409    6.666667
38435    6.666667
39452    6.625000
5114     6.590639
33486    6.514822
41195    6.500000
28891    6.480433
31964    6.397420
26055    6.396514
11741    6.395692
1575     6.356803
164      6.356502
813      6.325506
317      6.291209
34240    6.271122
Name: rating, dtype: float64

In [28]:
ratings_data.groupby('anime_id').rating.count().sort_values(ascending=False)[0:20] #lo más votados

anime_id
1535     3608
16498    3326
11757    3216
5114     3034
4224     3028
6547     2960
1575     2940
9253     2939
20       2782
10620    2721
19815    2713
30276    2701
22319    2624
226      2603
2167     2513
5081     2497
20507    2486
6746     2440
199      2436
32281    2431
Name: rating, dtype: int64

In [29]:
ratings_data.groupby('anime_id').rating.count().sort_values()[0:15] # hay varios animes con un solo review

anime_id
48492    1
38129    1
38128    1
38127    1
38126    1
38125    1
38124    1
38123    1
38122    1
38131    1
41707    1
38119    1
38118    1
38117    1
38099    1
Name: rating, dtype: int64

In [54]:
ratings_data.isna().sum()

user_id             0
anime_id            0
rating              0
watching_status     0
watched_episodes    0
dtype: int64

In [26]:
#nombre de los animes en el top 30
for id in top30.index:
    print(anime_data[anime_data.MAL_ID == id]['Name'])

15949    Ling Jian Zun 2nd Season
Name: Name, dtype: object
16837    Inochi no Tabekata
Name: Name, dtype: object
15400    Dramaturgy
Name: Name, dtype: object
2656    Code Geass: Hangyaku no Lelouch R2
Name: Name, dtype: object
14963    Shingeki no Kyojin Season 3 Part 2
Name: Name, dtype: object
176    Sen to Chihiro no Kamikakushi
Name: Name, dtype: object
11308    Kimi no Na wa.
Name: Name, dtype: object
1393    Death Note
Name: Name, dtype: object
10451    One Punch Man
Name: Name, dtype: object
11624    Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...
Name: Name, dtype: object
7449    Shingeki no Kyojin
Name: Name, dtype: object
15399    Nonsense Bungaku
Name: Name, dtype: object
404    Howl no Ugoku Shiro
Name: Name, dtype: object
9886    Koe no Katachi
Name: Name, dtype: object
17199    Gotcha!
Name: Name, dtype: object
16638    Wushen Zhuzai
Name: Name, dtype: object
14908    Yao Shen Ji Zhi Ying Yao Pian
Name: Name, dtype: object
15608    Itte.
Name: Name, dtype: object
3971

In [40]:
# Train Test split

train = ratings_data[(ratings_data.watching_status == 2) | (ratings_data.watching_status == 4)]
test = ratings_data[~(ratings_data.watching_status == 2) & ~(ratings_data.watching_status == 4)]

# Pivot the data to get it at the user-item level

train_pivot = train.pivot(index='user_id',columns='anime_id',values='rating')
watched = {}

for user in train_pivot.index:
  cols = train_pivot.loc[user].isna()
  watched[user] = train_pivot.loc[user,~cols].sort_values(ascending = False)


train_pivot.fillna(0, inplace = True)
test_pivot = test.pivot(index='user_id',columns='anime_id',values='rating').fillna(0)

cols_intersect = list( set(train_pivot.columns).intersection(set(test_pivot.columns))    )

train_pivot = train_pivot.loc[:,cols_intersect]
test_pivot = test_pivot.loc[:,cols_intersect]

test_pivot = test_pivot[test_pivot.index.isin(train_pivot)]

In [41]:
train_pivot.shape, test_pivot.shape

((4932, 14138), (185, 14138))

In [42]:
#Dictionaries for reference

user_dict = {}
anime_dict = {}

for i,user in enumerate(train_pivot.index):
  user_dict[user] = i

for i,anime in enumerate(test_pivot.columns):
  anime_dict[anime] = i

### Modelo

In [43]:
# Convert the DataFrame into a sparse matrix

train_csr = csr_matrix(train_pivot)
test_csr = csr_matrix(test_pivot)

# Split the data into training and testing sets

#train, test = train_test_split(pivot, test_size=0.2)

# Initialize the model
model = LightFM(learning_rate=0.03,
                no_components=100)

# Fit the model on the training data
model.fit(train_csr, epochs=10, verbose = True)

#with open('/content/drive/My Drive/Colab Notebooks/TP3/model_20k.pickle', 'rb') as file:
 #   model = pickle.load(file)

Epoch: 100%|██████████| 10/10 [00:27<00:00,  2.76s/it]


In [44]:
def user_recomendation(user, matrix_interaccion = train_pivot, model = model, user_dict = user_dict, n_predictions = 35):

  n_users, n_items = matrix_interaccion.shape
  item_ids = np.arange(n_items)

  user_x = user_dict[user]

  pred = model.predict(user_ids = user_x, item_ids = item_ids)

  rec = pd.Series(pred, index = matrix_interaccion.columns)

  #rec.sort_values(ascending = False, inplace = True)

  #rec = rec[:n_predictions]

  return rec

In [45]:
%%time

recoms = {}
recoms_ordered = {}

for user in train_pivot.index:
  r = user_recomendation(user) # We get full recomendations for each user
  #r = r[~r.index.isin(watched[user].index)]

  r_ordered = r.sort_values(ascending = False)
  #recoms[user] = {}
  recoms_ordered[user] = {}
  #recoms[user]["anime_ids"] = r.index.to_list()
  #recoms[user]["anime_scores"] = r.to_list()
  recoms_ordered[user]["anime_ids"] = r_ordered.index.to_list()
  recoms_ordered[user]["anime_scores"] = r_ordered.to_list()

CPU times: user 30.1 s, sys: 1.08 s, total: 31.1 s
Wall time: 31.3 s


In [46]:
# We translate the dict to a DataFrame

recoms_df = pd.DataFrame(recoms_ordered).T
recoms_df.index.name = "user"
recoms_df.reset_index(inplace = True)
recoms_df

,user,anime_ids,anime_scores
0,91,"[1535, 16498, 11757, 6547, 20, 30276, 5114, 42...","[4.479499816894531, 4.378915309906006, 4.37342..."
1,237,"[1535, 16498, 11757, 6547, 30276, 20, 4224, 51...","[5.5181074142456055, 5.435733318328857, 5.4301..."
2,329,"[1535, 16498, 11757, 6547, 20, 30276, 4224, 51...","[5.01694393157959, 4.926326751708984, 4.920782..."
3,389,"[1535, 16498, 11757, 6547, 30276, 4224, 20, 51...","[6.774643421173096, 6.709085464477539, 6.70343..."
4,393,"[1535, 16498, 11757, 20, 6547, 30276, 5114, 15...","[3.780616521835327, 3.6678061485290527, 3.6625..."
...,...,...,...
4927,353143,"[1535, 16498, 11757, 20, 6547, 30276, 5114, 15...","[3.5450196266174316, 3.430161476135254, 3.4247..."
4928,353218,"[1535, 16498, 11757, 6547, 20, 30276, 5114, 15...","[4.324477672576904, 4.222820281982422, 4.21727..."
4929,353275,"[1535, 16498, 11757, 20, 6547, 30276, 5114, 15...","[3.6002659797668457, 3.486544609069824, 3.4811..."
4930,353290,"[1535, 16498, 11757, 6547, 20, 30276, 5114, 42...","[4.570755481719971, 4.4750752449035645, 4.4694..."


In [47]:
# We now need to check the ideals for each user:

ideals = {}

for user in test_pivot.index:
  ideals[user] = {}
  d = test[test.user_id == user]
  ideals[user]["anime_ids"] = d.anime_id.to_list()
  ideals[user]["anime_scores"] = d.rating.to_list()

In [23]:
#ideal_recomms = test.sort_values(by=["user_id", "rating"], ascending=False)\
#                  .groupby(["user_id"], as_index=False)\
#                  .agg({"anime_id": "unique"}).head()
#ideal_recomms.head()

In [48]:
ideals_df = pd.DataFrame(ideals).T
ideals_df.index.name = "user_id"
ideals_df.reset_index(inplace = True)
ideals_df

,user_id,anime_ids,anime_scores
0,91,"[21995, 9919, 22789, 23333, 1735, 5530, 21881,...","[9, 8, 7, 5, 9, 9, 9, 9, 8, 8, 0, 0, 0, 0, 0, ..."
1,237,"[39184, 40935, 34662, 22297, 5114, 11061, 3972...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, ..."
2,329,"[2904, 1535, 227, 5114, 4039, 3701, 3572, 3673...","[0, 8, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 8, 0, ..."
3,389,"[41433, 820, 41911, 934, 41006, 20899, 40748, ...","[0, 0, 0, 0, 0, 7, 0, 5, 0, 4, 9, 7, 3, 4, 0, ..."
4,393,"[60, 3299, 3470, 1860, 2985, 1696, 4722]","[8, 0, 0, 0, 0, 0, 0]"
...,...,...,...
180,41574,"[36064, 33025, 42030, 31931, 10417, 389, 20715...","[9, 0, 0, 0, 0, 0, 0, 0, 0]"
181,41834,"[36649, 32983, 31229, 39491, 14289, 32998, 219...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
182,41842,"[35180, 33486, 34451, 36038, 36023, 9919, 21, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
183,42224,"[38408, 40010, 38256, 10165, 32603, 40028, 379...","[8, 10, 0, 9, 8, 1, 8, 6, 8, 0, 0, 0, 7, 6, 0,..."


In [49]:
# We can merge the ideals and the recoms to check the MAP

df_map = ideals_df.merge(recoms_df, how="left", left_on="user_id", right_on="user", suffixes = ["_ideals", "_recoms"])[["user_id","anime_ids_ideals","anime_ids_recoms"]]
df_map.columns = ["user_id","ideals","recoms"]
df_map.head()

,user_id,ideals,recoms
0,91,"[21995, 9919, 22789, 23333, 1735, 5530, 21881,...","[1535, 16498, 11757, 6547, 20, 30276, 5114, 42..."
1,237,"[39184, 40935, 34662, 22297, 5114, 11061, 3972...","[1535, 16498, 11757, 6547, 30276, 20, 4224, 51..."
2,329,"[2904, 1535, 227, 5114, 4039, 3701, 3572, 3673...","[1535, 16498, 11757, 6547, 20, 30276, 4224, 51..."
3,389,"[41433, 820, 41911, 934, 41006, 20899, 40748, ...","[1535, 16498, 11757, 6547, 30276, 4224, 20, 51..."
4,393,"[60, 3299, 3470, 1860, 2985, 1696, 4722]","[1535, 16498, 11757, 20, 6547, 30276, 5114, 15..."


In [50]:
df_map = df_map[df_map['ideals'].apply(lambda x: len(x) > 19)]
df_map.shape

(146, 3)

In [51]:
# And now we calculate MAP

aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideals", "recoms"]].values:
  if isinstance(label,list):
    n = len(pred) # cantidad de elementos recomendados
    arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1
    rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
    tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
    denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
    ap = (tp / denom).sum() / len(label) # average precision
    aps.append(ap)

In [52]:
pd.isna(label)

array([False, False, False, ..., False, False, False])

In [53]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.01056


In [19]:
# If we instead want to check NDCG Score

recoms_full_df = pd.DataFrame(recoms).T
recoms_full_df.index.name = "user"
recoms_full_df.reset_index(inplace = True)
recoms_full_df

,user


In [20]:
ideals_full = {}

for user in test_pivot.index:
  ideals_full[user] = {}
  d_full = test_pivot.loc[user]
  ideals_full[user]["anime_ids"] = d_full.index.to_list()
  ideals_full[user]["anime_scores"] = d_full.to_list()

In [21]:
ideals_full_df = pd.DataFrame(ideals_full).T
ideals_full_df.index.name = "user_id"
ideals_full_df.reset_index(inplace = True)
ideals_full_df

,user_id,anime_ids,anime_scores
0,106,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,163,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,391,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,495,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,518,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
156,40708,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
157,40763,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
158,40901,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
159,41401,"[1, 32772, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [55]:
#df_ndcg = ideals_full_df.merge(recoms_full_df, how="left", left_on="user_id", right_on="user", suffixes = ["_ideals", "_recoms"])[["user_id","anime_scores_ideals","anime_scores_recoms"]]
#df_ndcg.columns = ["user_id","ideals","recoms"]
#df_ndcg.head()

In [ ]:
#for i in df_ndcg.index:
#  df_ndcg.loc[i]